In [19]:
from jqdata import *
from jqlib.technical_analysis import *
from jqfactor import get_factor_values
from jqfactor import winsorize
from jqfactor import standardlize
from jqfactor import neutralize
import datetime
import pandas as pd
import numpy as np
import pickle
from six import StringIO

from sklearn.model_selection import train_test_split
from sklearn import metrics

import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

## Data Preparation

## 1. Get Trading Dates

- In order to acquire factor data at the end of each month, we need to get a list of the ending date of each month.

- input parameters
    - period: period_type {'W':week,'M':month,'Q':Quarter'}
    - start_date
    - end_date

- output: return ending date of each month between start_date and end_date

We choose 2010-01-01 as start_date and 2018-01-01 as end_date. 

In [20]:
#get the datelist of specific period 'W、M、Q'
def get_period_date(period,start_date, end_date):
    
    #stock code here is not important because we just want to identify the trading day
    stock_data = get_price('000001.XSHE',start_date,end_date,'daily',fields=['close'],panel=False)
    
    #record the trading days of each period 
    stock_data['date']=stock_data.index

    #resample data to the period bin and take the last data as the value of the bin
    #price/volume 
    period_stock_data = stock_data.resample(period).last()
    date = period_stock_data.index
    pydate_array = date.to_pydatetime()
    
    date_only_array = np.vectorize(lambda s: s.strftime('%Y-%m-%d'))(pydate_array)
    date_only_series = pd.Series(date_only_array)
    
    start_date = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    start_date =start_date-datetime.timedelta(days=1)
    start_date = start_date.strftime("%Y-%m-%d")
    
    date_list = date_only_series.values.tolist()
    date_list.insert(0,start_date)
    
    return date_list

#eg
#np.array(get_period_date('M','2010-01-01', '2018-01-01'))

## 2. Get Stocks Codes

- Available stock pool: zjw I64 I65
- Filter stocks 
    - rule out ST stocks
    - rule out stocks listed less than 3 months from current date(including new, delisted and suspended stocks)

In [21]:

def detect_stop(stocks,current_date,n=30*3):
    
    stockList=[]
    current_date = datetime.datetime.strptime(current_date, "%Y-%m-%d")
    
    for stock in stocks:
        start_date = get_security_info(stock).start_date
        if start_date < (current_date - datetime.timedelta(days=n)).date():
            stockList.append(stock)
    return stockList


def get_stock(cur_date):
    
    #get stock pool
    stockList = get_industry_stocks('I64')+get_industry_stocks('I65')
        
    #rule out ST
    st_data = get_extras('is_st',stockList, count = 1,end_date=cur_date)
    stockList = [stock for stock in stockList if not st_data[stock][0]]
    
    #rule out stocks listed less than 3 months from the begin_date
    #rule out delisted/suspended
    stockList = detect_stop(stockList,cur_date)
    
    return stockList
#eg
#get_stock('000300.XSHG','2017-06-01')

## 3. Feature Selection and Processing

- Feature Selection
    - On the last trading day of each month, calculate the factors exposures as the original features.These factors have been proved effecitve in the literature we find.

- Feature Preprocessing
    1. winsorize:
        - limiting extreme values in the data to reduce the effect of possibly spurious outliers. 
        - Realized by $\textit{winsorize()}$ provided by JoinQuant API.
    
    2. fill Nan: 
        - Replace Nans with the mean of the industry the stock belongs to. 
        - Realized by $\textit{replace_nan_indu()}$
    
    3. neutralize the market cap of the industry:
        - When we are using factors in the pool to evaluate stocks, there will be problems that these factors will be affected by some factors outside the pool, which will generate results under expectation.
        - For instance,P/B is highly related with market cap, so if we're using the P/B ratio before market cap neutralization, the stocks we choose will be mostly those with those with specific market cap feature.
        - Secondly, P/B ratio of those "sunrise" and "sunset" industry are generally similar respectively, meaning that industry also has impact on the the financial indicators, and we would very likely to get some results with unnecessary preferrences.
        - To address the 2 problems above, we are using neutralizing by industry and market cap: Perform a linear regression the factors value on the industry dummy variable and log of market cap, take the residuals as the new factor exposures.
        - Realized by $\textit{neutralize()}$ provided by JoinQuant API.
    
    4. standardlization:
        - Measures of factors values are diversed, so we need to standardlized them.
        - Realized by $\textit{standardlize()}$ provided by JoinQuant API.

In [22]:
#given stock, get corresponding industry
#industries are categorized by "zjw"
def get_industry_name(stocks_dict, value):
    return [k for k, v in stocks_dict.items() if value in v]

#deal with missing values
def replace_nan_indu(factor_data,stockList,industry_code,date):
    #first replace Nan with the mean of the industry the stock belongs to 
    #there will still be Nans, then replace them with the mean of all stocks
    
    stocks_dict = {}
    data_temp = pd.DataFrame(index=industry_code,columns=factor_data.columns)
    
    for i in industry_code:
        temp = get_industry_stocks(i, date)
        stocks_dict[i] = list(set(temp).intersection(set(stockList)))
        data_temp.loc[i]=mean(factor_data.loc[stocks_dict[i],:])
        
    #first replace Nan with the mean of the industry the stock belongs to
    for factor in data_temp.columns:
        null_industry = list(data_temp.loc[pd.isnull(data_temp[factor]),factor].keys())
        for i in null_industry:
            data_temp.loc[i,factor] = mean(data_temp[factor])
            
        #there will still be Nans, then replace them with the mean of all stocks
        null_stock=list(factor_data.loc[pd.isnull(factor_data[factor]),factor].keys())
        for i in null_stock:
            industry=get_industry_name(stocks_dict, i)
            if industry:
                factor_data.loc[i,factor]=data_temp.loc[industry[0],factor] 
            else:
                factor_data.loc[i,factor]=mean(factor_data[factor])
    return factor_data

#data preprocessing
def data_preprocessing(factor_data,stockList,industry_code,date):
    
    #winsorize data to eliminate of the effect of outliers
    factor_data = winsorize(factor_data, qrange=[0.05,0.95],inf2nan=False, axis=0)
    
    #deal with missing values
    factor_data = replace_nan_indu(factor_data,stockList,industry_code,date)
    
    #neutralize the data to avoid the effect that industry put on related factor
    factor_data = neutralize(factor_data, how=['zjw', 'market_cap'], date=date, axis=0)
    
    #standardize the data
    factor_data = standardlize(factor_data,axis=0)
    
    return factor_data

#acquire the factors data for all stocks on time 'date'
def get_factor_data(stock,date):
    
    data=pd.DataFrame(index=stock)
    q = query(valuation,balance,cash_flow,income,indicator).filter(valuation.code.in_(stock))
    df = get_fundamentals(q, date)
    
    df['market_cap']=df['market_cap']*100000000
    
    factor_data=get_factor_values(stock,['roe_ttm','roa_ttm','total_asset_turnover_rate',\
                               'net_operate_cash_flow_ttm','net_profit_ttm',\
                              'cash_to_current_liability','current_ratio',\
                             'gross_income_ratio','non_recurring_gain_loss',\
                            'operating_revenue_ttm','net_profit_growth_rate',\
                            'sharpe_ratio_20','sharpe_ratio_60','sharpe_ratio_120',\
                            'net_debt','EBIT','intangible_asset_ratio','DEGM',\
                            'roic_ttm','total_asset_growth_rate',\
                            'operating_revenue_growth_rate','net_operate_cashflow_growth_rate'],\
                            end_date=date,count=1)
    
    factor=pd.DataFrame(index=stock)
    for i in factor_data.keys():
        factor[i]=factor_data[i].iloc[0,:]
    df.index = df['code']
    del df['code'],df['id']
    
    #combine
    df=pd.concat([df,factor],axis=1)
    
    # Profitability indicator
    df['net_assets']=df['total_assets']-df['total_liability']
    
    data['roic_ttm']=df['roic_ttm']
    data['log_market_cap']=np.log(df['market_cap'])
    data['pe_ratio']=df['net_profit_ttm']/df['market_cap']
    data['pb_ratio']=1/df['pb_ratio']
    data['ps_ratio']=1/df['ps_ratio']
    data['roe_ttm']=df['roe_ttm']
    data['roe_q']=df['roe']
    data['roa_ttm']=df['roa_ttm']
    data['roa_q']=df['roa']
    data['gross_profit_margin_ttm']=df['gross_income_ratio']
    data['gross_profit_margin_q']=df['gross_profit_margin']
    data['profit_margin_q']=df['adjusted_profit']/df['operating_revenue']

    #financial indicator - leverage
    data['financial_leverage']=df['total_assets']/df['net_assets']
    data['debt_to_equity_ratio']=df['total_non_current_liability']/df['net_assets']
    data['cash_ratio']=df['cash_to_current_liability']
    data['current_ratio']=df['current_ratio']
    
    #growth indicators
    data['intangible_asset_ratio']=df['intangible_asset_ratio']
    data['total_asset_growth_rate']=df['total_asset_growth_rate']
    data['profit_margin_growth_rate']=df['DEGM']
    data['net_profit_growth_rate']=df['net_profit_growth_rate']
    data['asset_turnover_ttm']=df['total_asset_turnover_rate']
    data['asset_turnover_q']=df['operating_revenue']/df['total_assets']
    data['operating_revenue_growth_rate']=df['operating_revenue_growth_rate']
    data['net_operate_cf_growth_rate']=df['net_operate_cashflow_growth_rate']
    
    #momentum indicators
    ##contrarian
    stock_close=get_price(stock, count = 60*20+1, end_date=date, frequency='daily', fields=['close'],panel=False)['close']
    data['contrarian_1m']=stock_close.iloc[-1]/stock_close.iloc[-20]-1
    data['contrarian_3m']=stock_close.iloc[-1]/stock_close.iloc[-60]-1
    data['contrarian_6m']=stock_close.iloc[-1]/stock_close.iloc[-120]-1
    data['contrarian_12m']=stock_close.iloc[-1]/stock_close.iloc[-240]-1

    ##get turnover
    data_turnover_ratio=pd.DataFrame()
    data_turnover_ratio['code']=stock
    trade_days=list(get_trade_days(end_date=date, count=240*2))
    for i in trade_days:
        q = query(valuation.code,valuation.turnover_ratio).filter(valuation.code.in_(stock))
        temp = get_fundamentals(q, i)
        data_turnover_ratio=pd.merge(data_turnover_ratio, temp,how='left',on='code')
        data_turnover_ratio=data_turnover_ratio.rename(columns={'turnover_ratio':i})
    data_turnover_ratio=data_turnover_ratio.set_index('code').T  
    
    ##daily average turnover ratio in the closet N months
    data['stock_turnover_1m']=mean(data_turnover_ratio.iloc[-20:])
    data['stock_turnover_3m']=mean(data_turnover_ratio.iloc[-60:])
    data['stock_turnover_6m']=mean(data_turnover_ratio.iloc[-120:])
    data['stock_turnover_12m']=mean(data_turnover_ratio.iloc[-240:])
    
    data['bias_stock_turnover_1m']=mean(data_turnover_ratio.iloc[-20:])/mean(data_turnover_ratio)-1
    data['bias_stock_turnover_3m']=mean(data_turnover_ratio.iloc[-60:])/mean(data_turnover_ratio)-1
    data['bias_stock_turnover_6m']=mean(data_turnover_ratio.iloc[-120:])/mean(data_turnover_ratio)-1
    data['bias_stock_turnover_12m']=mean(data_turnover_ratio.iloc[-240:])/mean(data_turnover_ratio)-1

    #risk indicators
    ##volatility 
    stock_pchg=stock_close.pct_change().iloc[1:]
    data['volatility_of_return_1m']=stock_pchg.iloc[-20:].std()
    data['volatility_of_return_3m']=stock_pchg.iloc[-60:].std()
    data['volatility_of_return_6m']=stock_pchg.iloc[-120:].std()
    data['volatility_of_return_12m']=stock_pchg.iloc[-240:].std()
    
    ##sharpe ratio
    data['sharpe_ratio_20d']=df['sharpe_ratio_20']
    data['sharpe_ratio_60d']=df['sharpe_ratio_60']
    data['sharpe_ratio_120d']=df['sharpe_ratio_120']
      
    return data

#### Write Pickle File (DONOT RUN THIS SECTION)

In [ ]:
peroid = 'M' #month
start_date = '2010-01-01'
end_date = '2018-01-01'

#industry code 
industry_code = ['I64','I65']

#get trading dates 
dateList = get_period_date(peroid,start_date, end_date)

factor_init_data = {}
factor_final_data = {}


for date in dateList:
    
    print(date)
        
    stockList=get_stock(date)

    factor_init_data[date] = get_factor_data(stockList,date)
    factor_final_data[date] = data_preprocessing(factor_init_data[date],stockList,industry_code,date)
    
content = pickle.dumps(factor_final_data) 
write_file('factor_data.pkl', content, append=False)

In [23]:
factor_read=open('factor_data.pkl','rb')
factors=pickle.load(factor_read)

## 4. Data Overview

Data Type: dict
- key: date (the date of the last trading day of each month)
- value: the corresponding factors data of each period
    - value data format: dataframe
    - index: stock codes 
    - columns: factors

In [24]:
factors['2009-12-31'].head()

,roic_ttm,log_market_cap,pe_ratio,pb_ratio,ps_ratio,roe_ttm,roe_q,roa_ttm,roa_q,gross_profit_margin_ttm,gross_profit_margin_q,profit_margin_q,financial_leverage,debt_to_equity_ratio,cash_ratio,current_ratio,intangible_asset_ratio,total_asset_growth_rate,profit_margin_growth_rate,net_profit_growth_rate,asset_turnover_ttm,asset_turnover_q,operating_revenue_growth_rate,net_operate_cf_growth_rate,contrarian_1m,contrarian_3m,contrarian_6m,contrarian_12m,stock_turnover_1m,stock_turnover_3m,stock_turnover_6m,stock_turnover_12m,bias_stock_turnover_1m,bias_stock_turnover_3m,bias_stock_turnover_6m,bias_stock_turnover_12m,volatility_of_return_1m,volatility_of_return_3m,volatility_of_return_6m,volatility_of_return_12m,sharpe_ratio_20d,sharpe_ratio_60d,sharpe_ratio_120d
000503.XSHE,-0.512500,-1.120879,-0.388257,-1.077097,-1.709625,-0.692879,-1.075033,-0.733108,-1.302710,-0.375977,-0.528139,-1.156109,-1.214253,-1.976323,0.588487,-0.425839,-1.113684,0.850188,-1.040031,-1.695110,-0.893657,-1.091178,0.771176,-0.717801,NaN,-1.129151,-1.129151,-1.129151,0.380998,0.674747,0.238648,-0.055786,0.689594,1.206733,0.553964,-0.238196,NaN,-1.129151,-1.129151,-1.129151,0.458526,-0.150085,0.499285
000606.XSHE,-0.055995,-0.195735,-0.053428,-0.116181,-0.005641,-0.062522,-0.068488,-0.064927,-0.075975,-0.107351,-0.109912,-0.087277,-0.085980,-0.041941,-0.038178,-0.104063,-0.059347,-0.029840,0.006779,-0.027945,-0.034239,-0.043841,-0.017284,0.008792,NaN,-0.196095,-0.196095,-0.196095,-0.111905,-0.124008,-0.124254,-0.127046,-0.016064,-0.016436,-0.044982,-0.064437,NaN,-0.196095,-0.196095,-0.196095,0.005883,-0.036394,-0.054728
000676.XSHE,-1.274996,-2.107013,-1.572833,-1.491951,0.184579,-1.423820,-1.946296,-0.878980,-1.575495,-1.118237,-1.461499,-1.662347,-0.519623,-1.809495,-0.668470,-0.522201,-0.662788,-0.965518,-0.695595,-0.679368,-0.633371,-0.449688,-0.449884,-0.276353,NaN,-2.144547,-2.144547,-2.144547,-2.332846,-1.680949,-1.757855,-1.574726,-0.830873,-0.036228,-0.499266,0.035731,NaN,-2.144547,-2.144547,-2.144547,0.076652,-1.010742,-0.564930
000835.XSHE,-0.055995,-0.195735,-0.053428,-0.116181,-0.005641,-0.062522,-0.068488,-0.064927,-0.075975,-0.107351,-0.109912,-0.087277,-0.085980,-0.041941,-0.038178,-0.104063,-0.059347,-0.029840,0.006779,-0.027945,-0.034239,-0.043841,-0.017284,0.008792,NaN,-0.196095,-0.196095,-0.196095,-0.111905,-0.124008,-0.124254,-0.127046,-0.016064,-0.016436,-0.044982,-0.064437,NaN,-0.196095,-0.196095,-0.196095,0.005883,-0.036394,-0.054728
002072.XSHE,-0.991742,-0.047729,-2.519291,-1.350659,-0.212915,-1.649759,-1.109104,-1.123769,-0.742310,0.036543,0.061779,-0.890730,1.421403,0.091122,-0.029984,-0.229745,-0.170111,0.720764,-1.813131,-3.422662,-0.483676,-0.439951,0.026448,0.336944,NaN,0.041457,0.041457,0.041457,-0.517916,0.269485,0.231060,0.445673,-1.084964,0.282963,0.596060,1.321173,NaN,0.041457,0.041457,0.041457,-2.049653,-2.002098,-0.505727


## 5. Adding Labels

In [25]:
period='M'
start_date = '2010-01-01'
end_date = '2018-02-01'
industry_code = ['I64','I65']
dateList=get_period_date(period,start_date, end_date)

In [26]:
factor_data=pd.DataFrame()

for date in dateList[:-2]:
    
    temp=factors[date]  
    if factor_data.empty:
        factor_data=temp
    else:
        factor_data=factor_data.append(temp)
dic={}
for column in list(factor_data.columns[factor_data.isnull().sum() > 0]):
    mean_val = factor_data[column].mean()
    dic[column]=mean_val

In [27]:
#train_test_split 
from sklearn.model_selection import train_test_split

date_train, date_test=train_test_split(dateList,test_size=0.4,shuffle=False)

In [28]:
#train set +y

train_data=pd.DataFrame()

for date in date_train:
    
    traindf=factors[date]
    stockList=list(traindf.index)
    
    #acquire return data
    data_close=get_price(stockList,date,dateList[dateList.index(date)+1],'1d','close')['close']
    traindf['pchg']=data_close.iloc[-1]/data_close.iloc[0]-1

    #replaceNan
    for column in list(factor_data.columns[factor_data.isnull().sum() > 0]):
        traindf[column].fillna(dic[column],inplace=True)
        
    #select first 30% (label=1) and last 30% (label=-1),rule out noises in between
    traindf=traindf.sort_values('pchg',ascending=False)
    traindf=traindf.iloc[:len(traindf['pchg'])//10*3,:].append(traindf.iloc[len(traindf['pchg'])//10*7:,:])
    traindf['label']=list(traindf['pchg'].apply(lambda x:1 if x>np.mean(list(traindf['pchg'])) else -1))  
    #traindf['label']=list(traindf['pchg'].apply(lambda x:1 if x>average_annual_return else -1))
    if train_data.empty:
        train_data=traindf
    else:
        train_data=train_data.append(traindf)  

In [18]:
train_data.to_csv("train_set.csv")

In [29]:
test_data={}

for date in date_test[:-2]:
    
    testdf=factors[date]
    stockList=list(testdf.index)
    
    data_close=get_price(stockList,date,dateList[dateList.index(date)+1],'1d','close')['close']
    testdf['pchg']=data_close.iloc[-1]/data_close.iloc[0]-1

    #replaceNan
    for column in list(factor_data.columns[factor_data.isnull().sum() > 0]):
        testdf[column].fillna(dic[column],inplace=True)


    testdf=testdf.sort_values('pchg',ascending=False)
    testdf=testdf.iloc[:len(testdf['pchg'])//10*3,:].append(testdf.iloc[len(testdf['pchg'])//10*7:,:])
    testdf['label']=list(testdf['pchg'].apply(lambda x:1 if x>np.mean(list(testdf['pchg'])) else -1)) 
    
    #testdf['label']=list(testdf['pchg'].apply(lambda x:1 if x>average_annual_return else -1))
    test_data[date]=testdf

In [16]:
content = pickle.dumps(test_data) 
write_file('test_set.pkl', content, append=False)

1806172